In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.ml.classification import LogisticRegression
from pyspark.sql import functions as when
from handyspark import *
from matplotlib import pyplot as plt
%matplotlib inline
from pyspark.ml.evaluation import BinaryClassificationEvaluator,MulticlassClassificationEvaluator
from pyspark.sql.functions import col

In [ ]:
spark = SparkSession.builder.appName('BigDataProject').getOrCreate()

In [ ]:
data = spark.read.option("multiLine","true").csv("G:\My Drive\Sem 2\Big Data Analytics\Big Data Project\WA_Fn-UseC_-Telco-Customer-Churn.csv",inferSchema=True,header=True)

### Check out the data

In [ ]:
data.describe

<bound method DataFrame.describe of DataFrame[customerID: string, gender: string, SeniorCitizen: int, Partner: string, Dependents: string, tenure: int, PhoneService: string, MultipleLines: string, InternetService: string, OnlineSecurity: string, OnlineBackup: string, DeviceProtection: string, TechSupport: string, StreamingTV: string, StreamingMovies: string, Contract: string, PaperlessBilling: string, PaymentMethod: string, MonthlyCharges: double, TotalCharges: string, Churn: string]>

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer
indexers = [StringIndexer(inputCol=column, outputCol=column+"_index").fit(data) for column in list(set(data.columns)-set(['customerID'])) ]

pipeline = Pipeline(stages=indexers)
df_r = pipeline.fit(data).transform(data)

#df_r.show(5)

In [ ]:
"""
df_r = df_r.drop('customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure', 'PhoneService', 'MultipleLines',
 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies',
 'Contract', 'PaperlessBilling', 'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn')
df_r.show()
"""

"\ndf_r = df_r.drop('customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure', 'PhoneService', 'MultipleLines',\n 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies',\n 'Contract', 'PaperlessBilling', 'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn')\ndf_r.show()\n"

In [ ]:
#data.select(df_basket1.colRegex("`(Item)+?.+`")).show()
selected_df = [s for s in df_r.columns if '_index' in s]#+['index']
df_encoded=df_r.select(selected_df)
#df_encoded.show(5)

In [ ]:
from pyspark.ml.feature import VectorAssembler

In [ ]:
assembler = VectorAssembler(inputCols=df_encoded.columns,outputCol='features')
#assembler = VectorAssembler(inputCols=['Dependents_index', 'tenure_index', 'TotalCharges_index', 'TechSupport_index', 'InternetService_index', 'PaymentMethod_index', 'Partner_index', 'Contract_index', 'SeniorCitizen_index', 'OnlineBackup_index', 'PaperlessBilling_index', 'StreamingTV_index', 'MultipleLines_index', 'StreamingMovies_index', 'PhoneService_index', 'MonthlyCharges_index', 'OnlineSecurity_index','DeviceProtection_index', 'gender_index'],outputCol='features')

In [ ]:
output = assembler.transform(df_encoded)
#output.show(1)

In [ ]:
final_data = output.select('features','churn_index')

In [ ]:
#final_data=data.select("gender", "SeniorCitizen", "Partner", "Dependents", "tenure", "PhoneService", "MultipleLines", "InternetService", "OnlineSecurity", "OnlineBackup", "DeviceProtection", "TechSupport", "StreamingTV", "StreamingMovies", "Contract", "PaperlessBilling", "PaymentMethod", "MonthlyCharges","TotalCharges","churn")
final_data.show(5)

+--------------------+-----------+
|            features|churn_index|
+--------------------+-----------+
|(20,[0,1,6,10,11,...|        0.0|
|(20,[5,6,7,8,9,10...|        0.0|
|(20,[1,5,6,9,10,1...|        1.0|
|(20,[5,6,7,8,9,10...|        0.0|
|(20,[5,6,13,17,19...|        1.0|
+--------------------+-----------+
only showing top 5 rows



In [ ]:
def evalmetrics(predf):
    predf = predf.select('features','churn_index','probability','prediction','rawPrediction')
    evaluator = BinaryClassificationEvaluator(rawPredictionCol='rawPrediction', labelCol='churn_index')
    mcEvaluator = MulticlassClassificationEvaluator(metricName="accuracy", labelCol='churn_index',)
    
    #print confusion Matrix
    print("Confusion Matrix:")
    cf=predf.crosstab('churn_index', 'prediction').show()
    #cf.show()
    
    
    # Accuarcy and Area and ROC
    auroc = evaluator.evaluate(predf, {evaluator.metricName: "areaUnderROC"})
    print("Area under ROC Curve: {:.4f}".format(auroc))
    print(f"Accuracy : {mcEvaluator.evaluate(predf)}")

### Test Train Split

In [ ]:
train_churn,test_churn = final_data.randomSplit([0.6,0.4])

### Fit the model (Logistic Regression)

In [ ]:
lr_churn = LogisticRegression(labelCol='churn_index', featuresCol="features",maxIter=10, regParam=0.5)
fitted_churn_model = lr_churn.fit(train_churn)

In [ ]:
training_sum = fitted_churn_model.summary
#training_sum.predictions.show(5)#.describe()
#predDF_train=training_sum.predictions.select("churn_index","prediction")

predDF_train_lr = fitted_churn_model.transform(train_churn)
#predDF_train = predDF_train.select('features','churn_index','probability','prediction','rawPrediction')

In [ ]:
print("Logistic Regression Model Evalution (Training Data)")
evalmetrics(predDF_train_lr)

Logistic Regression Model Evalution (Training Data)
Confusion Matrix:
+----------------------+----+---+
|churn_index_prediction| 0.0|1.0|
+----------------------+----+---+
|                   1.0| 286|818|
|                   0.0|3111|  0|
+----------------------+----+---+

Area under ROC Curve: 0.9975
Accuracy : 0.93214709371293


### Evaluate results

In [ ]:
predDF_test_lr = fitted_churn_model.transform(test_churn)
print("Logistic Regression Model Evalution (Test Data)")
evalmetrics(predDF_test_lr)

Logistic Regression Model Evalution (Test Data)
Confusion Matrix:
+----------------------+----+---+
|churn_index_prediction| 0.0|1.0|
+----------------------+----+---+
|                   1.0| 229|536|
|                   0.0|2063|  0|
+----------------------+----+---+

Area under ROC Curve: 0.9982
Accuracy : 0.919024045261669


# Naive Bayes Model Model

In [ ]:
from pyspark.ml.classification import NaiveBayes
#from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
# create the trainer and set its parameters
nb = NaiveBayes(smoothing=1.0, modelType="multinomial",featuresCol='features', labelCol='churn_index')

# train the model
model = nb.fit(train_churn)

In [ ]:
predDF_train_nb = model.transform(train_churn)
print("Naive Bayes Model Model Evalution (Train Data)")
evalmetrics(predDF_train_nb)

Naive Bayes Model Model Evalution (Train Data)
Confusion Matrix:
+----------------------+----+----+
|churn_index_prediction| 0.0| 1.0|
+----------------------+----+----+
|                   1.0| 491| 613|
|                   0.0|2007|1104|
+----------------------+----+----+

Area under ROC Curve: 0.5440
Accuracy : 0.6215895610913404


In [ ]:
predDF_test_nb = model.transform(test_churn)
print("Naive Bayes Model Model Evalution (Test Data)")
evalmetrics(predDF_test_nb)

Naive Bayes Model Model Evalution (Test Data)
Confusion Matrix:
+----------------------+----+---+
|churn_index_prediction| 0.0|1.0|
+----------------------+----+---+
|                   1.0| 317|448|
|                   0.0|1347|716|
+----------------------+----+---+

Area under ROC Curve: 0.5179
Accuracy : 0.6347241867043847
